In [ ]:
# analyze_training.py
import numpy as np
import matplotlib.pyplot as plt
import struct
import yfinance as yf
import prepare_data

In [ ]:
tickers = prepare_data.get_tickers(n_total=140)
print(f"Selected {len(tickers)} tickers\n")
    
    # 2. Download closing prices (15 years)
print("Downloading 15 years of closing prices...")
data = yf.download(tickers, period="15y", progress=True)
prices = data['Close']  

In [ ]:
prices

In [ ]:


def load_parameters(filename):
    """Charge les paramètres depuis le fichier binaire"""
    with open(filename, 'rb') as f:
        N = struct.unpack('i', f.read(4))[0]
        K = struct.unpack('i', f.read(4))[0]
        
        A = np.frombuffer(f.read(N * N * 4), dtype=np.float32).reshape(N, N)
        pi = np.frombuffer(f.read(N * 4), dtype=np.float32)
        mu = np.frombuffer(f.read(N * K * 4), dtype=np.float32).reshape(N, K)
        Sigma = np.frombuffer(f.read(N * K * K * 4), dtype=np.float32).reshape(N, K, K)
        
    return {'N': N, 'K': K, 'A': A, 'pi': pi, 'mu': mu, 'Sigma': Sigma}

def plot_training_history(csv_file):
    """Plot la courbe de convergence"""
    data = np.loadtxt(csv_file, delimiter=',', skiprows=1)
    iterations = data[:, 0]
    log_likelihood = data[:, 1]
    
    plt.figure(figsize=(10, 6))
    plt.plot(iterations, log_likelihood, 'b-', linewidth=2)
    plt.xlabel('Iteration')
    plt.ylabel('Log-Likelihood')
    plt.title('EM Training Convergence')
    plt.grid(True, alpha=0.3)
    plt.savefig('training_convergence.png', dpi=150)
    plt.show()

def analyze_parameters(params):
    """Analyse les paramètres appris"""
    print("=== ANALYSE DES PARAMÈTRES ===\n")
    
    print("Distribution initiale π :")
    print(params['pi'])
    print()
    
    print("Matrice de transition A :")
    print(params['A'])
    print()
    
    print("Moyennes μ (premiers 5 assets) :")
    print(params['mu'][:, :5])
    print()

 
    
    # Persistance des états
    persistence = np.diag(params['A'])
    print("Persistance des états (diagonale de A) :")
    for i, p in enumerate(persistence):
        print(f"  État {i}: {p:.4f}")

if __name__ == '__main__':
    params = load_parameters('trained_params.bin')
    analyze_parameters(params)
    plot_training_history('training_history.csv')

In [ ]:
# Matrice de transition
A = params['A']
print("Matrice de transition A:")
print(A)

# Visualiser
import seaborn as sns
plt.figure(figsize=(8, 6))
sns.heatmap(A, annot=True, fmt='.3f', cmap='YlOrRd', 
            xticklabels=[f'État {i}' for i in range(3)],
            yticklabels=[f'État {i}' for i in range(3)])
plt.title('Probabilités de transition entre régimes')
plt.savefig('transition_matrix.png', dpi=150)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt



# Plot des moyennes par état
fig, axes = plt.subplots(3, 1, figsize=(12, 10))

for state in range(3):
    axes[state].bar(range(len(params['mu'][state])), params['mu'][state])
    axes[state].set_title(f'État {state} - Rendements moyens (μ)')
    axes[state].set_xlabel('Asset index')
    axes[state].set_ylabel('Mean return')
    axes[state].axhline(y=0, color='r', linestyle='--', alpha=0.3)
    axes[state].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('states_interpretation.png')
plt.show()

# Statistiques par état
for state in range(3):
    mean_return = np.mean(params['mu'][state])
    volatility = np.sqrt(np.mean(np.diag(params['Sigma'][state])))
    print(f"État {state}:")
    print(f"  Rendement moyen: {mean_return:.4f}")
    print(f"  Volatilité moyenne: {volatility:.4f}")
    print(f"  Persistance: {params['A'][state, state]:.4f}")
    print()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Charger les états
states = pd.read_csv('decoded_states.csv')

# Reconstituer les dates (15 ans de données boursières)
# Environ 252 jours ouvrés par an × 15 = 3780 jours
# Mais tu as 3647 jours → commence probablement vers début 2010
start_date = pd.Timestamp('2010-12-23')  
dates = pd.bdate_range(start=start_date, periods=len(states))
states['date'] = dates

# Définir les couleurs et labels
colors = {0: 'green', 1: 'red', 2: 'orange'}
labels = {0: 'Calme/Haussier', 1: 'Crise/Bear', 2: 'Normal/Mixte'}

# Timeline
fig, ax = plt.subplots(figsize=(16, 4))
for state in range(3):
    mask = states['state'] == state
    ax.scatter(states[mask]['date'], [state]*mask.sum(), 
              c=colors[state], label=labels[state], alpha=0.6, s=1)

ax.set_yticks([0, 1, 2])
ax.set_yticklabels(labels.values())
ax.set_xlabel('Date')
ax.set_ylabel('Régime de marché')
ax.set_title('Régimes de marché identifiés par HMM sur 15 ans (100+ assets)')
ax.legend()
ax.grid(True, alpha=0.2)
plt.tight_layout()
plt.savefig('regime_timeline_dated.png', dpi=200)
plt.show()

# Identifier COVID
covid_start = 2175
covid_end = 2218
print(f"Période de crise la plus longue (COVID?):")
print(f"  Du {states.iloc[covid_start]['date'].date()} au {states.iloc[covid_end]['date'].date()}")
print(f"  Durée: {covid_end - covid_start + 1} jours")

# Compter les transitions par an
states['year'] = states['date'].dt.year
transitions_per_year = states.groupby('year').apply(
    lambda x: (x['state'].diff() != 0).sum()
)
print("\nNombre de changements de régime par année:")
print(transitions_per_year)

In [61]:
import numpy as np
import yfinance as yf
import pandas as pd
from hmmlearn import hmm
import struct
import os
import matplotlib.pyplot as plt

In [65]:


# ==========================================
# CONFIGURATION
# ==========================================
T = 2000    # Assez long pour converger
N = 3       # 3 États (ex: Bull, Bear, Sideways)
K = 4       # 4 Features
filename_prefix = "golden_data"

np.random.seed(42) # Reproductibilité totale

# ==========================================
# 1. GÉNÉRATION DE LA VÉRITÉ TERRAIN (Ground Truth)
# ==========================================
print("1. Génération des données synthétiques...")

# Vrais paramètres (ce qu'on espère retrouver environ)
true_means = np.array([
    [0.0, 0.0, 0.0, 0.0],       # Etat 0: Neutre
    [2.0, 2.0, -1.0, 1.0],      # Etat 1: Positif
    [-2.0, -2.0, 1.0, 2.0]      # Etat 2: Négatif
])

# Covariances : Identité (données sphériques, faciles à apprendre)
true_covars = np.tile(np.eye(K), (N, 1, 1))

# Transitions : On reste longtemps dans chaque état (0.95 sur diag)
true_transmat = np.array([
    [0.95, 0.025, 0.025],
    [0.025, 0.95, 0.025],
    [0.025, 0.025, 0.95]
])

true_startprob = np.array([1.0, 0.0, 0.0]) # Commence toujours état 0

# Instance du modèle générateur
gen_model = hmm.GaussianHMM(n_components=N, covariance_type="full")
gen_model.startprob_ = true_startprob
gen_model.transmat_ = true_transmat
gen_model.means_ = true_means
gen_model.covars_ = true_covars

# Génération des échantillons
X, Z = gen_model.sample(T)

# ==========================================
# 2. NORMALISATION (CRITIQUE POUR HMM)
# ==========================================
# On centre et réduit les données pour éviter les underflows
print("2. Normalisation Z-Score...")
mean_X = np.mean(X, axis=0)
std_X = np.std(X, axis=0)
X_norm = (X - mean_X) / std_X

# ==========================================
# 3. INITIALISATION DU MODÈLE A APPRENDRE
# ==========================================
# On crée un modèle "Vierge" qu'on va entraîner
print("3. Configuration du modèle d'apprentissage...")

model_train = hmm.GaussianHMM(n_components=N, covariance_type="full", n_iter=20, tol=1e-4, verbose=True, init_params="")

# Initialisation "Intelligente mais pas parfaite" (K-Means like ou Random propre)
# Pour comparer avec C++, on fixe ces valeurs initiales manuellement ici.

# A. Moyennes initiales : Petits randoms autour de 0
init_means = np.random.normal(0, 0.5, (N, K)) 

# B. Covariances initiales : Identité un peu élargie (Stabilité)
init_covars = np.tile(np.eye(K), (N, 1, 1)) * 1.0

# C. Transition : Uniforme + bruit
init_transmat = np.ones((N, N)) / N
# On bruite légèrement pour casser la symétrie
init_transmat += np.random.uniform(0, 0.01, (N, N))
# Normalisation des lignes
init_transmat = init_transmat / init_transmat.sum(axis=1, keepdims=True)

# D. Pi
init_pi = np.ones(N) / N

# On injecte ces params dans le modèle hmmlearn
model_train.startprob_ = init_pi
model_train.transmat_ = init_transmat
model_train.means_ = init_means
model_train.covars_ = init_covars

# ==========================================
# 4. ENTRAÎNEMENT PYTHON (REFERENCE)
# ==========================================
print("\n--- Lancement HMMLearn (Fit) ---")
# fit() modifie le modèle en place
model_train.fit(X_norm)

print(f"\n[Python Reference] Final Log-Likelihood: {model_train.score(X_norm)}")
print(f"[Python Reference] Converged in {model_train.monitor_.iter} iterations")

print("\nMoyennes Apprises (doivent ressembler aux Vrais Paramètres normalisés) :")
print(model_train.means_)

# ==========================================
# 5. EXPORT POUR C++
# ==========================================
print(f"\n5. Export des fichiers binaires vers le dossier '{filename_prefix}'...")
os.makedirs(filename_prefix, exist_ok=True)

def write_bin(name, array):
    path = os.path.join(filename_prefix, name)
    with open(path, 'wb') as f:
        f.write(array.astype(np.float32).tobytes())
    print(f" -> {path}")

# On exporte les données NORMALISÉES
write_bin("obs.bin", X_norm)

# On exporte les paramètres INITIAUX (ceux AVANT le fit)
# Attention : model_train contient maintenant les params appris !
# Il faut réexporter les variables init_... définies plus haut.
write_bin("pi_init.bin", init_pi)
write_bin("A_init.bin", init_transmat)
write_bin("mu_init.bin", init_means)
write_bin("sigma_init.bin", init_covars)

# Metadata
with open(os.path.join(filename_prefix, "dims.txt"), "w") as f:
    f.write(f"{T} {N} {K}")

print("\nTerminé.")

1. Génération des données synthétiques...
2. Normalisation Z-Score...
3. Configuration du modèle d'apprentissage...

--- Lancement HMMLearn (Fit) ---


         1  -11655.81337390             +nan
         2   -9636.55570279   +2019.25767111
         3   -9216.02088424    +420.53481855
         4   -9069.00510648    +147.01577776
         5   -9038.31421777     +30.69088871
         6   -9035.30652881      +3.00768896
         7   -9034.63229837      +0.67423045
         8   -9034.08539731      +0.54690105
         9   -9033.53670113      +0.54869619
        10   -9032.94830065      +0.58840048
        11   -9032.26968048      +0.67862017



[Python Reference] Final Log-Likelihood: -8205.089839669481
[Python Reference] Converged in 20 iterations

Moyennes Apprises (doivent ressembler aux Vrais Paramètres normalisés) :
[[-1.05541529 -1.06753276  0.78908883  0.7563364 ]
 [ 0.03644832  0.03847312 -0.03373706 -0.77042952]
 [ 1.03097152  1.04107688 -0.76382181  0.0634685 ]]

5. Export des fichiers binaires vers le dossier 'golden_data'...
 -> golden_data/obs.bin
 -> golden_data/pi_init.bin
 -> golden_data/A_init.bin
 -> golden_data/mu_init.bin
 -> golden_data/sigma_init.bin

Terminé.


        12   -9031.41717130      +0.85250918
        13   -9030.23606904      +1.18110226
        14   -9028.40576887      +1.83030017
        15   -9025.17227475      +3.23349412
        16   -9018.47112449      +6.70115026
        17   -9001.45516890     +17.01595560
        18   -8946.01354986     +55.44161904
        19   -8740.09639440    +205.91715546
        20   -8332.91913348    +407.17726092
